# **Taller 06: Bases de datos Vectoriales**

Nombres: Rossy Armendariz, Alejandro Chavez, Wilmer Rivas.

## Parte 1: Recuperación con TF-IDF

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Leer el archivo CSV que contiene los datos de las películas
data = pd.read_csv('./data/wiki_movie_plots_deduped.csv')

# Filtrar solo las columnas "Title" y "Plot" necesarias para el análisis
selected_data = data[['Title', 'Plot']]

# Visualizar las primeras filas del DataFrame resultante
print(selected_data.head())

                              Title  \
0            Kansas Saloon Smashers   
1     Love by the Light of the Moon   
2           The Martyred Presidents   
3  Terrible Teddy, the Grizzly King   
4            Jack and the Beanstalk   

                                                Plot  
0  A bartender is working at a saloon, serving dr...  
1  The moon, painted with a smiling face hangs ov...  
2  The film, just over a minute long, is composed...  
3  Lasting just 61 seconds and consisting of two ...  
4  The earliest known adaptation of the classic f...  


In [2]:
# Configuración del vectorizador TF-IDF para procesar el texto
tfidf_vectorizer = TfidfVectorizer(stop_words='english')

# Generación de la matriz TF-IDF basada en la columna 'Plot'
tfidf_matrix = tfidf_vectorizer.fit_transform(selected_data['Plot'])

# Definición de la función para ejecutar búsquedas basadas en TF-IDF
def tfidf_search(query_text, results_count=5):
  
    # Transformar la consulta en vector TF-IDF
    query_vector = tfidf_vectorizer.transform([query_text])
    
    # Calcular la similitud entre la consulta y los documentos
    similarity_scores = cosine_similarity(query_vector, tfidf_matrix).flatten()
    
    # Seleccionar los índices de los documentos más relevantes
    best_match_indices = similarity_scores.argsort()[-results_count:][::-1]
    
    # Crear un DataFrame con los resultados
    top_results = selected_data.iloc[best_match_indices][['Title', 'Plot']].copy()
    top_results['Score'] = similarity_scores[best_match_indices]
    
    return top_results

# Ejemplo de consulta
search_query = "A young boy discovers magic"
search_results = tfidf_search(search_query)

# Mostrar los resultados de la búsqueda
print(search_results)

                         Title  \
34283  Fantastic Journey to OZ   
432              Grandma's Boy   
28716      72 Mile - Ek Pravas   
28379         Once Upon a Time   
34034         Once Upon A Time   

                                                    Plot     Score  
34283  The envious and power-hungry Urfin Jus wants t...  0.235492  
432    The grandma's boy is a timid coward who cannot...  0.232582  
28716  A young boy decides to escape from his boardin...  0.225330  
28379  A sage gives Rahul a book containing various c...  0.220203  
34034  A sage gives Rahul a book containing various c...  0.220203  


### Evaluación de resultados.
Los documentos recuperados contienen términos relacionados con young boy o temas similares como "boy", "timid", "escape", pero no necesariamente magic, lo que indica que TF-IDF priorizó coincidencias textuales exactas y no relaciones semántica para identificar términos relacionados o sinónimos. Además se visualiza que no existe una grnade diferencia entre las similitudes de los documentos.

## Parte 2: Recuperación con BM25

%pip install elasticsearch

In [3]:
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk
import pandas as pd

# Establecer conexión con el servidor Elasticsearch
elastic_client = Elasticsearch("http://localhost:9200")  # Incluyendo el esquema http

# Nombre del índice a manejar
index_name = "movie_plots"

# Verificar si el índice ya existe, y eliminarlo si es necesario
if elastic_client.indices.exists(index=index_name):
    elastic_client.indices.delete(index=index_name)
    print(f"Índice '{index_name}' eliminado correctamente.")

# Crear un índice nuevo en Elasticsearch
elastic_client.indices.create(index=index_name)
print(f"Índice '{index_name}' creado con éxito.")


Índice 'movie_plots' eliminado correctamente.
Índice 'movie_plots' creado con éxito.


In [4]:
# Función para cargar los datos en el índice existente
def cargar_datos():
    for fila, datos in selected_data.iterrows():  # Usamos el DataFrame filtrado
        yield {
            "_index": index_name,
            "_source": {
                "Indice": int(fila),  # Almacena el índice original convertido a un entero
                "Titulo": datos['Title'],
                "Sinopsis": datos['Plot']
            }
        }

# Enviar los documentos al índice
bulk(elastic_client, cargar_datos())
print("Proceso de indexación finalizado.")

Proceso de indexación finalizado.


In [5]:
# Función para realizar consultas con BM25
def search_bm25(query, top_k=5):
    # Construir el cuerpo de la consulta
    query_body = {
        "query": {
            "match": {
                "Sinopsis": query  # Cambiar "Plot" por "Sinopsis" si así se indexaron los datos
            }
        },
        "size": top_k  # Limitar el número de resultados
    }

    # Ejecutar la consulta en Elasticsearch
    response = elastic_client.search(index=index_name, body=query_body)

    # Crear una lista para almacenar los resultados
    results = []
    for hit in response['hits']['hits']:
        # Asegurarse de manejar el caso en que 'Indice' no exista
        results.append({
            "Index": hit['_source'].get('Indice', None),  # Usar .get() para evitar errores
            "Title": hit['_source'].get('Titulo', "Unknown Title"),
            "Plot": hit['_source'].get('Sinopsis', "No Plot Available"),
            "Score": hit['_score']  # Puntuación de relevancia
        })

    # Convertir la lista en un DataFrame
    # Usar .set_index solo si "Index" tiene valores válidos
    df_results = pd.DataFrame(results)
    if 'Index' in df_results.columns and df_results['Index'].notnull().all():
        df_results = df_results.set_index("Index")
    
    return df_results

# Ejemplo de ejecución
query = "A young boy discovers magic"  # Texto de búsqueda
results = search_bm25(query)  # Recuperar los documentos relevantes
print(results)  # Mostrar los resultados


                                  Title  \
Index                                     
12820         Sabrina the Teenage Witch   
24430                             Devta   
432                       Grandma's Boy   
32935  Magic Serpent !The Magic Serpent   
33953       Mary and the Witch's Flower   

                                                    Plot      Score  
Index                                                                
12820  The movie centers around Sabrina Sawyer, who i...  12.786088  
24430  This film narrated the story of a king who los...  11.225356  
432    The grandma's boy is a timid coward who cannot...  10.938396  
32935  The Oumi Kingdom, ruled peacefully by Lord Oga...  10.933238  
33953  A fire burns as workers struggle to control it...  10.827491  


### Comparación de resultados.
Se puede observar que BM25 proporciona resultados que se alinean mejor con el tema de la consulta ("A young boy discovers magic"). Por ejemplo, títulos como Sabrina the Teenage Witch y Mary and the Witch's Flower tienen asociaciones claras con magia y juventud, lo cual está más alineado con la intención de la consulta. Mientras que TF-IDF, algunos casos incluye resultados como 72 Mile - Ek Pravas, que no parecen tan relacionados con magia, lo que puede deberse a su dependencia en la frecuencia relativa de términos sin considerar contexto. Por otro lado, los puntajes de BM25 son más variables (12.78 para el documento más relevante frente a ~10.8 para otros), lo que indica una priorización más clara de los documentos más relevantes.
En TF-IDF, los puntajes son más uniformes (~0.22-0.23), lo que sugiere que su capacidad para distinguir relevancia es más limitada.

## Parte 3: Recuperación con FAISS

In [6]:
%pip install sentence-transformers faiss-cpu


[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
from sentence_transformers import SentenceTransformer
import numpy as np
from tqdm import tqdm  # Para mostrar barra de progreso
import os
import faiss

os.environ["HF_HUB_DISABLE_SYMLINKS_WARNING"] = "1"

In [10]:
from sentence_transformers import SentenceTransformer
import numpy as np
from tqdm import tqdm  # Para mostrar una barra de progreso

# Modelo preentrenado para generar embeddings
modelo_embeddings = SentenceTransformer('all-MiniLM-L6-v2')

# Configuración del tamaño de lotes
tamano_lote = 64

# Extraer la columna "Plot" del DataFrame 'selected_data' como lista
sinopsis = selected_data['Plot'].tolist()

# Inicializar una lista vacía para almacenar los embeddings generados
vectores_embeddings = []

print("Generando vectores de embeddings...")

# Procesar las sinopsis en lotes y generar los vectores
for inicio in tqdm(range(0, len(sinopsis), tamano_lote), desc="Procesando lotes"):
    lote = sinopsis[inicio:inicio + tamano_lote]  # Crear un lote a partir de la lista
    lote_embeddings = modelo_embeddings.encode(lote)  # Generar embeddings para el lote
    vectores_embeddings.extend(lote_embeddings)  # Agregar los embeddings al resultado final

# Convertir los vectores de embeddings en un arreglo NumPy
vectores_embeddings = np.array(vectores_embeddings)

print(f"Dimensiones del arreglo de embeddings: {vectores_embeddings.shape}")

Generando vectores de embeddings...


Procesando lotes: 100%|██████████| 546/546 [36:59<00:00,  4.06s/it]

Dimensiones del arreglo de embeddings: (34886, 384)


In [11]:
# Crear índice FAISS para búsqueda basada en vectores
dimension_vectores = vectores_embeddings.shape[1]  # Dimensión de cada vector de embedding
indice_faiss = faiss.IndexFlatL2(dimension_vectores)  # Crear índice usando la métrica de distancia Euclidiana (L2)

# Agregar los embeddings al índice FAISS
indice_faiss.add(vectores_embeddings)  # Cargar los vectores en el índice

# Verificar el total de vectores almacenados en el índice
print(f"Total de vectores en el índice FAISS: {indice_faiss.ntotal}")


Total de vectores en el índice FAISS: 34886


In [14]:
# Función para realizar búsquedas con FAISS basada en embeddings
def realizar_busqueda_faiss(consulta, num_resultados=5):
    # Generar el embedding para la consulta
    vector_consulta = modelo_embeddings.encode([consulta])

    # Consultar el índice FAISS para obtener los vecinos más cercanos
    distancias, indices_vecinos = indice_faiss.search(vector_consulta, num_resultados)

    # Extraer los datos relevantes del DataFrame original
    resultados = selected_data.iloc[indices_vecinos.flatten()][['Title', 'Plot']].copy()
    
    # Ajustar nombres de columnas y agregar la distancia calculada
    resultados.rename(columns={'Title': 'Titulo', 'Plot': 'Sinopsis'}, inplace=True)
    resultados['Distancia'] = distancias.flatten()

    return resultados

# Ejemplo práctico de uso
consulta_prueba = "A young boy discovers magic"
resultados_prueba = realizar_busqueda_faiss(consulta_prueba)

# Imprimir los resultados obtenidos
print(resultados_prueba)


                        Titulo  \
13065          The Midas Touch   
17232                  Sleight   
21101           The Mirror Boy   
7329   The Boy and the Pirates   
28728            Dusari Goshta   

                                                Sinopsis  Distancia  
13065  Drama about a 12-year-old boy who fantasises a...   0.965923  
17232  A young street magician named Bo (Jacob Latimo...   1.077003  
21101  The film tells the uplifting story of a young ...   1.107963  
7329   A boy, Jimmy Warren, living along the coast in...   1.161079  
28728  A young boy from the lower caste resorts to pe...   1.170947  


### Comparación de resultados
FAISS nos da documentos que tienen una relación más semántica con la consulta "A young boy discovers magic". Por ejemplo, The Midas Touch y Sleight son altamente relevantes, debido a que combinan temas de magia y juventud. Existe una mejor comprensión mejor al enfocarse en el contexto general y no solamente en coincidencias exactas. Las distancias entre el embedding de la consulta y los documentos son claras (0.96 a 1.17). Esto indica un orden jerárquico efectivo en la similitud semántica.
Sin embargo, FAISS presenta un resultado superior al identificar relaciones semánticas más profundas. Los resultados de BM25 están más relacionados con palabras clave exactas, mientras que FAISS entiende el concepto general. BM25 mejora sobre TF-IDF al ajustar la relevancia considerando la longitud de los documentos y la saturación de términos. 

## Parte 4: Recuperación con ChromaDB

In [2]:
import torch
print(torch.cuda.is_available())
print(torch.version.cuda)
print(torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU found")


True
12.4
NVIDIA GeForce RTX 3050 Ti Laptop GPU


Configuración de ChromaDB y embeddings

In [3]:
import pandas as pd
import chromadb
from chromadb.utils import embedding_functions
import torch

# Verificar si la GPU está disponible
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Usando el dispositivo: {device}")

# Cargar el archivo CSV de películas
movies_df = pd.read_csv("./wiki_movie_plots_deduped.csv")

# Filtrar campos necesarios
movies_df = movies_df[movies_df['Plot'].notna()]  # Eliminar filas con Plot vacío
documents = movies_df['Plot'].tolist()  # Lista de las tramas
titles = movies_df['Title'].tolist()   # Lista de los títulos

# Configurar cliente Chroma
client = chromadb.Client()

# Crear la función de embeddings con GPU
embedding_function = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name="all-MiniLM-L6-v2",  # Modelo eficiente y rápido
    device=device  # Especificar el dispositivo (GPU o CPU)
)

# Crear colección con la función de embeddings configurada
collection = client.get_or_create_collection(
    name="movies_collection",
    embedding_function=embedding_function
)

# Generar IDs únicos para cada documento
ids = [f"movie_{i}" for i in range(len(movies_df))]

# Dividir los datos en lotes
batch_size = 5000  # Tamaño del lote (ajustar según sea necesario)
for i in range(0, len(documents), batch_size):
    batch_documents = documents[i:i+batch_size]
    batch_titles = titles[i:i+batch_size]
    batch_ids = ids[i:i+batch_size]
    
    # Agregar el lote a la colección
    collection.add(
        documents=batch_documents,  # Tramas del lote
        metadatas=[{"title": title} for title in batch_titles],  # Metadata del lote
        ids=batch_ids  # IDs únicos del lote
    )
    print(f"Lote {i // batch_size + 1} procesado exitosamente.")

print(f"Se han agregado {len(documents)} documentos y generado sus embeddings.")


Usando el dispositivo: cuda
Lote 1 procesado exitosamente.
Lote 2 procesado exitosamente.
Lote 3 procesado exitosamente.
Lote 4 procesado exitosamente.
Lote 5 procesado exitosamente.
Lote 6 procesado exitosamente.
Lote 7 procesado exitosamente.
Se han agregado 34886 documentos y generado sus embeddings.


Consulta aplicando ChromaDB

In [9]:
# Definir una consulta (puede ser una trama de película o cualquier texto)
query = "A young boy discovers "

# Realizar la consulta en la colección
results = collection.query(
    query_texts=[query],  # Pasar la consulta como texto
    n_results=5  # Número de resultados a devolver
)

# Mostrar los resultados
for i, document in enumerate(results['documents'][0]):  # Accedemos a la lista de documentos en el primer índice
    print(f"Resultado {i + 1}:")
    print(f"Trama: {document}")  # Solo mostramos la trama si no hay metadata disponible
    print("-" * 50)


Resultado 1:
Trama: An investigative thriller based on the search for a missing youngster.
--------------------------------------------------
Resultado 2:
Trama: A young boy from the lower caste resorts to petty thefts to make both ends meet after he lost his father at the early age. Once the boy realizes the importance of an education, he begins to improve his life and never looks back. Through diligence and dedication, he climbs the social and political ladder to success.
--------------------------------------------------
Resultado 3:
Trama: Thirteen-year-old Jesse is assigned a school project. A photographic self-portrait intended to portray one’s self without resorting to literal representation. Jesse lives with his parents, Sabi and Tim, in the lefty, middle class Toronto neighbourhood of Riverdale. A quiet and distant only-child with budding artistic aspirations, Jesse is inspired by the assignment to look for excitement and meaning in the world around him. Wielding a newly acqui

### Evaluación de los resultados

Los resultados de ChromaDB, TF-IDF, BM25, y FAISS muestran diferencias notables en cómo recuperan documentos. ChromaDB brinda tramas que combinan elementos de "magia" y "un niño joven", mostrando relaciones semánticas más profundas con la consulta. Por ejemplo, incluye temas como búsqueda científica o aventuras de jóvenes con trasfondo mágico. FAISS, al igual que ChromaDB, prioriza la semántica, pero sus resultados tienen una ligera variación al incluir documentos con contextos más amplios relacionados con la consulta. BM25 se centra más en términos exactos como "magic" y "boy", recuperando documentos altamente relevantes, aunque no tan ricos en contexto como los de ChromaDB. Por último, TF-IDF, aunque eficiente, es menos preciso al depender de la frecuencia de las palabras, incluyendo tramas menos relacionadas con "magia". Por tanto, ChromaDB y FAISS son más efectivos para consultas complejas y temáticas.

## Parte 5: Comparación de Resultados

1. Relevancia:

TF-IDF: Identificó documentos relacionados principalmente con palabras clave. Aunque incluyó títulos interesantes como "Fantastic Journey to OZ", algunos resultados como "72 Mile - Ek Pravas" fueron menos relevantes, evidenciando la falta de contexto semántico.

BM25: Mejoró los resultados al priorizar documentos con mayor densidad y ajuste por longitud, recuperando títulos más relacionados como "Sabrina the Teenage Witch". Pero sigue siendo dependiente de palabras clave, y además mostró mejor precisión que TF-IDF.

FAISS: Al utilizar embeddings, capturó relaciones semánticas más profundas. Los resultados como "The Midas Touch" y "Sleight" tuvieron una conexión mejor a la idea de "magia" y "niño joven". Sin embargo, requiere más recursos computacionales.

ChromaDB: Fue el enfoque más completo, combinando semántica con resultados organizados. Recuperó documentos que incluyen magia y temas científicos, mostrando una fuerte conexión temática con la query.

2. Ventajas y limitaciones:

TF-IDF: Rápido y sencillo, pero limitado en consultas complejas debido a su falta de semántica.
BM25: Equilibrado y efectivo para búsquedas basadas en texto, pero no captura relaciones abstractas entre términos.
FAISS: Excelente en semántica, identificando conceptos más allá de coincidencias textuales, pero requiere mayor capacidad de procesamiento.
ChromaDB: Combina almacenamiento persistente y semántica, increible para grandes volúmenes de datos, aunque necesita configuración más avanzada.

En conclusión, ChromaDB y FAISS son ideales para consultas complejas y relaciones semánticas, mientras que BM25 y TF-IDF son adecuados para consultas más directas. La elección dependerá del contexto y los recursos disponibles.